<h1 align="center"> Deep Learning gender from name - RNN LSTMs </h1>

In [0]:
import tensorflow as tf


In [0]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
import keras
from keras.layers import Dense, Activation, Dropout, Flatten, AveragePooling1D
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import pandas as pd
import numpy as np
import os

import unicodedata
import string


Using TensorFlow backend.


In [0]:
# tf.__version__

In [0]:
#parameters namelength = maxlen, labels = male, female
maxlen = 25
labels = 2

# **Data Preparation**

In [0]:

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

#print(unicodeToAscii('Ślusàrski fgdf'))

In [0]:
# f = open('./f-5000-names.txt').read().split(",")
# m = open('./m-5000-names.txt').read().split(",")


In [0]:
female = pd.read_csv('./Indian-Female-Names.csv')
male = pd.read_csv('./Indian-Male-Names.csv')


In [0]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')


#f = [''.join(filter(whitelist.__contains__, unicodeToAscii(str(i).lower()))) for i in f]
#m = [''.join(filter(whitelist.__contains__, unicodeToAscii(str(i).lower()))) for i in m]


  

female = [''.join(filter(whitelist.__contains__, unicodeToAscii(str(i).lower()))) for i in female.name.tolist()]
male = [''.join(filter(whitelist.__contains__, unicodeToAscii(str(i).lower()))) for i in male.name.tolist()]



#f = list(set(f) - set(m1))
#m = list(set(m) - set(f1))

f = ["a"]
m = ["sa"]

In [0]:
# while("   " in f):
#   f.remove("   ")
# while("   " in m):
#   m.remove("   ")

# while("   " in female):
#   female.remove("   ")

# while("   " in male):
#   male.remove("   ")



# while("  " in f):
#   f.remove("  ")


# while("  " in m):
#   m.remove("  ")


# while("  " in female):
#   female.remove("  ")


# while("  " in male):
#   male.remove("  ")



# while(" " in f):
#   f.remove(" ")


# while(" " in m):
#   m.remove(" ")


# while(" " in female):
#   female.remove(" ")


# while(" " in male):
#   male.remove(" ")
  
# while("" in f):
#   f.remove("")


# while("" in m):
#   m.remove("")


# while("" in female):
#   female.remove("")


# while("" in male):
#   male.remove("")








In [0]:
for i in range(len(female)):
  x = female[i]

  if x[:2] == 'mr' or x[:3] == 'ms':
    x = x[2:]
  elif x[:3] == 'mrs' or x[:3] == 'smt':
    x = x[3:]
  elif x[:4] == 'mohd' or x[:4] == 'shri':
    x = x[4:]
  
  while x[0]=="":
    x = x[1:]
  
  if len(x.split())>=2:
    x = x.split()[0]
  
  female[i] = x

for i in range(len(male)):
  x = male[i]

  if x[:2] == 'mr' or x[:3] == 'ms':
    x = x[2:]
  elif x[:3] == 'mrs' or x[:3] == 'smt':
    x = x[3:]
  elif x[:4] == 'mohd' or x[:4] == 'shri':
    x = x[4:]
  

  while x[0]=="":
    x = x[1:]
  
  if len(x.split())>=2:
    x = x.split()[0]
  
  male[i] = x

In [0]:
# for i in female:
#   i = " ".join(i.split())
# for i in male:
#   i = " ".join(i.split())

In [0]:
# for i in range(len(f)):


#   #print(f[i], i)
#   x = f[i].split()[:2]
  
  
#   if x[0] in ['mr', 'mrs', 'smt', 'mohd', 'shri', 'ms'] or len(x[0])<=2:
#     f[i] = str(x[1]) if len(x)>=2 else ''
#   else:
#     f[i] = str(x[0])

# for i in range(len(m)):


#   #print(m[i], i)
#   x = m[i].split()[:2]
  
  
#   if x[0] in ['mr', 'mrs', 'smt', 'mohd', 'shri', 'ms'] or len(x[0])<=2:
#     m[i] = str(x[1]) if len(x)>=2 else ''
#   else:
#     m[i] = str(x[0])

# for i in range(len(female)):


#   #print(female[i], i)
#   x = female[i].split()[:2]
  
  
#   if x[0] in ['mr', 'mrs', 'smt', 'mohd', 'shri', 'ms'] or len(x[0])<=2:
#     female[i] = str(x[1]) if len(x)>=2 else ''
#   else:
#     female[i] = str(x[0])

# for i in range(len(male)):


#   #print(male[i], i)
#   x = male[i].split()[:2]
  
  
#   if x[0] in ['mr', 'mrs', 'smt', 'mohd', 'shri', 'ms'] or len(x[0])<=2:
#     male[i] = str(x[1]) if len(x)>=2 else ''
#   else:
#     male[i] = str(x[0])

In [0]:
# fn1 = [] #unique names with majority use in gender
# mn1 = [] #unique names with majority use in gender
# cs = f + m
# for i in list(set(cs)):
#   if i=='':
#     continue
  
#   if f.count(i) >= m.count(i):
#     fn1.append(i)
#   else:
#     mn1.append(i)


In [0]:
fn = [] #unique names with majority use in gender ex. male.count('rahul') = 5 and female.count('rahul') = 1...hence gender is male
mn = [] #unique names with majority use in gender

combined_gender = female + male
for i in list(set(combined_gender)):
  if i=='':
    continue
  i = " ".join(i.split())
  if female.count(i) >= male.count(i):
    #i = " ".join(i.split())
    fn.append(i)
  else:
    mn.append(i)


In [0]:
# fn1 = list(set(fn1) - set(mn))
# mn1 = list(set(mn1) - set(fn))

In [0]:
female_names = fn #fn + fn1
male_names = mn #mn + mn1

len(female_names), len(male_names), len(set(female_names)), len(set(male_names))

(3372, 3758, 3172, 3657)

In [0]:
unique_female_names = list(set(female_names))
unique_male_names = list(set(male_names))

In [0]:
unique_male_names = unique_male_names[:len(unique_female_names)]

In [0]:
len(unique_female_names), len(unique_male_names)

(3372, 3372)

In [0]:
name_combines = unique_female_names + unique_male_names
len(name_combines)

6744

In [0]:
len(name_combines), len(set(name_combines)), len(unique_female_names), len(unique_male_names)



(6744, 6468, 3372, 3372)

In [0]:
#unique_male_names.index('mohit')

In [0]:
f = ["f"]*len(unique_female_names)
m = ["m"]*len(unique_male_names)
m_or_f = f+m  #f should be at first

In [0]:
data_dict = {'name': name_combines, 'm_or_f': m_or_f}

df = pd.DataFrame(data_dict)

df.head(20)


,name,m_or_f
0,tarunnam,f
1,gytri,f
2,sima,f
3,balalia,f
4,ilma,f
5,shiya,f
6,rubi,f
7,munmun,f
8,salama,f
9,gunjan,f


In [0]:
# df['label'] = df['m_or_f'].apply(lambda x: 1 if x == 'f' else 0)

In [0]:
# df = df[['name', 'label']].sample(frac=1)

In [0]:
# !pip install phonetics
# import phonetics
# df['vowel_ending'] = df['name'].apply(lambda x: x[-1].lower() in set('aeiouy'))
# df['last'] = df['name'].apply(lambda x: x[-1].lower())
# #df['last_allit'] = df['name'].apply(lambda x: x[-1] == x[-2])
# df['phonetics'] = df['name'].apply(lambda x: phonetics.metaphone(x))

In [0]:
export_csv = df.to_csv ('gender_data.csv', index = None)

In [0]:
input_dataframe = pd.read_csv("gender_data.csv",header=None)
input_dataframe.columns = ['name','m_or_f']
input_dataframe['namelen']= [len(str(i)) for i in input_dataframe['name']]
input1 = input_dataframe[(input_dataframe['namelen'] >= 2) ]

In [0]:
input1.head()

,name,m_or_f,namelen
0,name,m_or_f,4
1,tarunnam,f,8
2,gytri,f,5
3,sima,f,4
4,balalia,f,7


In [0]:
input1 = input1.iloc[1:]
#input1['vowel_ending'] = input1['vowel_ending'].apply(lambda x: x.lower())


In [0]:
# input1['name'] = input1['name'] + input1['phonetics']
# input1.head()

In [0]:
names = input_dataframe['name']
gender = input_dataframe['m_or_f']
#phonetics = input1['phonetics']
vocab = list(' '.join([unicodeToAscii(str(i)) for i in names]))
#vocab1 = list(' '.join([unicodeToAscii(str(i)) for i in phonetics]))
vocab = set(vocab)
vocab.add('END')
# vocab.add('VOWEL')
# vocab.add('LAST')

len_vocab = len(vocab)

In [0]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'a', 'i', 'k', 'v', 'c', 'h', 'n', 'l', 'r', 'f', 's', 'q', 'END', 'x', 't', ' ', 'd', 'j', 'm', 'g', 'y', 'u', 'o', 'e', 'b', 'z', 'p', 'w'}
vocab length is  28
length of input is  6733


In [0]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [0]:
# char_index['0'] = 29
# print(char_index)


In [0]:
#train test split
msk = np.random.rand(len(input1)) < 0.8
train = input1[msk]
test = input1[~msk]     

In [0]:
len(train), len(test)

(5358, 1375)

In [0]:
def set_flag(i):
    tmp = np.zeros(len(vocab));
    tmp[i] = 1
    return(tmp)

In [0]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    i = " ".join(i.split())
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)


In [0]:
np.asarray(train_X).shape

(5358, 25, 28)

In [0]:
# set_flag(3)

#### modify the code above to also convert each index to one-hot encoded representation

In [0]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    tmp = np.array(tmp)
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [0]:
# np.asarray(train_X).shape

In [0]:
# np.asarray(train_Y).shape

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [0]:
#build the model: 2 stacked LSTM

hidden_nodes = int(2/3 * (maxlen * len_vocab))
print('Build model...')
model = Sequential()

model.add(LSTM(hidden_nodes, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))

model.add(LSTM(hidden_nodes, return_sequences=False))
model.add(Dropout(0.2))

#model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

#from keras import optimizers
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 25, 466)           922680    
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 466)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 466)               1739112   
_________________________________________________________________
dropout_2 (Dropout)          (None, 466)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 934       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 2,662,726
Trainable params: 2,662,726
Non-trainable params: 0
____________________________________________

In [0]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    tmp = np.array(tmp)
    shape = tmp.shape
    tmp.reshape([1, shape[0], shape[1]])
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    

In [0]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(1375, 25, 28)
(1375, 2)


In [0]:
train_X = np.array(train_X)
train_Y = np.array(train_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=70,validation_data=(test_X, test_Y))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  





Train on 5358 samples, validate on 1375 samples
Epoch 1/70





5358/5358 [==============================] - 12s 2ms/step - loss: 0.6978 - acc: 0.4983 - val_loss: 0.6948 - val_acc: 0.4975
Epoch 2/70
5358/5358 [==============================] - 1s 220us/step - loss: 0.6942 - acc: 0.4944 - val_loss: 0.6927 - val_acc: 0.5025
Epoch 3/70
5358/5358 [==============================] - 1s 218us/step - loss: 0.6912 - acc: 0.5468 - val_loss: 0.6815 - val_acc: 0.5905
Epoch 4/70
5358/5358 [==============================] - 1s 221us/step - loss: 0.6760 - acc: 0.5911 - val_loss: 0.6818 - val_acc: 0.6320
Epoch 5/70
5358/5358 [==============================] - 1s 222us/step - loss: 0.6833 - acc: 0.5750 - val_loss: 0.6781 - val_acc: 0.6131
Epoch 6/70
5358/5358 [==============================] - 1s 223us/step - loss: 0.6626 - acc: 0.6297 - val_loss: 0.6724 - val_acc: 0.5505
Epoch 7/70
5358/5358 [==============================] - 1s 219us/step - loss: 0.6462 - acc: 0.6269 - val_loss: 0.6593 - val_acc: 

In [0]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

1375/1375 [==============================] - 1s 1ms/step
Test score: 0.4413424301364205
Test accuracy: 0.8123636358001015


In [0]:
name = ['arun', 'ravi', 'shiva', 'tulsi', 'bhanu', 'simran', 'varun', 'lovely', 'sumit', 'madhu', 'sakti', 'saroj', 'aditya', 'himanshu']
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))

x = 0
for i in pred:
  if i[0]>0.7 or i[1]>0.7:
    pr = 'male' if np.argmax(i) == 0 else 'female'
    print(name[x], pr, i)
  else:
    print(name[x],'Find from image', i)
  x+=1
  

arun Find from image [0.38626096 0.6137391 ]
ravi Find from image [0.64587504 0.35412496]
shiva female [0.14806013 0.85193986]
tulsi Find from image [0.49932063 0.5006794 ]
bhanu Find from image [0.6831102  0.31688985]
simran Find from image [0.6803287  0.31967124]
varun male [0.8610457 0.1389543]
lovely female [0.25212765 0.7478724 ]
sumit male [0.93268543 0.06731457]
madhu Find from image [0.4680304 0.5319696]
sakti female [0.1194437 0.8805563]
saroj male [0.9320922  0.06790784]
aditya Find from image [0.6679771  0.33202288]
himanshu male [0.8850499  0.11495011]


In [0]:
X=[]
q = 1
name = [str(i)[0:maxlen] for i in test.name]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
for  i in range(len(pred)):
    pr = 'male' if np.argmax(pred[i]) == 0 else 'female'
    ac = 'male' if np.argmax(test_Y[i]) == 0 else 'female'
    if pr != ac and (pred[i][0]>0.7 or pred[i][1]>0.7):
      q+=1
      print ("{}, actual: {}, pred: {}".format(name[i], ac, pr))

print(q)

mariyam, actual: female, pred: male
ishawar, actual: female, pred: male
raghuvinder, actual: female, pred: male
jioty, actual: female, pred: male
kamar, actual: female, pred: male
dolat, actual: female, pred: male
sharjprit, actual: female, pred: male
amie, actual: female, pred: male
jugan, actual: female, pred: male
chatru, actual: female, pred: male
santosh, actual: female, pred: male
batloon, actual: female, pred: male
ataullah, actual: female, pred: male
binu, actual: female, pred: male
rafat, actual: female, pred: male
nandu, actual: female, pred: male
tausif, actual: female, pred: male
lakhvinder, actual: female, pred: male
rajenderi, actual: female, pred: male
ifteshyam, actual: female, pred: male
munesh, actual: female, pred: male
viren, actual: female, pred: male
durgash, actual: female, pred: male
afroz, actual: female, pred: male
paramjeet, actual: female, pred: male
juneb, actual: female, pred: male
bishnu, actual: female, pred: male
ahshan, actual: female, pred: male
shm, 

In [0]:
# name=["sanju","simran","madhu", "amit"]
# X=[]
# trunc_name = [i[0:maxlen] for i in name]
# for i in trunc_name:
#     tmp = [set_flag(char_index[j]) for j in str(i)]
#     for k in range(0,maxlen - len(str(i))):
#         tmp.append(set_flag(char_index["END"]))
#     X.append(tmp)
# pred=model.predict(np.asarray(X))
# pred

In [0]:
#save our model and data
model.save('gender_model',overwrite=True)
# train.to_csv("train_split.csv")
# test.to_csv("test_split.csv")

In [0]:
from keras.models import load_model
new_model = load_model('./gender_model')

In [0]:
train_X.shape, test_X.shape, test_Y.shape, train_Y.shape

((5108, 25, 28), (1225, 25, 28), (1225, 2), (5108, 2))

In [0]:
X=[]
q = 1
name = [str(i)[0:maxlen] for i in test.name]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=new_model.predict(np.asarray(X))
for  i in range(len(pred)):
    pr = 'male' if np.argmax(pred[i]) == 0 else 'female'
    ac = 'male' if np.argmax(test_Y[i]) == 0 else 'female'
    if pr != ac and (pred[i][0]>0.7 or pred[i][1]>0.7):
      q+=1
      print ("{}, actual: {}, pred: {}".format(name[i], ac, pr))

print(q)

mehvish, actual: female, pred: male
tarunnam, actual: female, pred: male
hasnain, actual: female, pred: male
nancy, actual: female, pred: male
jasvinder, actual: female, pred: male
aazadijully, actual: female, pred: male
siwam, actual: female, pred: male
alis, actual: female, pred: male
fool, actual: female, pred: male
bsram, actual: female, pred: male
daulat, actual: female, pred: male
jioty, actual: female, pred: male
maaya, actual: female, pred: male
kajal, actual: female, pred: male
saveroon, actual: female, pred: male
used, actual: female, pred: male
parmjeet, actual: female, pred: male
nirvat, actual: female, pred: male
samsher, actual: female, pred: male
sudesh, actual: female, pred: male
sabenoortamanna, actual: female, pred: male
ishawar, actual: female, pred: male
ekamjeet, actual: female, pred: male
nasim, actual: female, pred: male
awshin, actual: female, pred: male
rukhsar, actual: female, pred: male
taran, actual: female, pred: male
rupender, actual: female, pred: male
ru